In [152]:
import pandas as pd
import numpy as np
from time import sleep
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os


In [153]:
#Librerias de Selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [154]:
#Hacemos que no muestre los warnings
import warnings
warnings.filterwarnings("ignore")


In [155]:
def abrir_web(ciudad, pagina, fecha1, fecha2):

    driver = webdriver.Chrome()

    url = f"https://www.civitatis.com/es/{ciudad}/?pagina={pagina}&fromDate={fecha1}&toDate={fecha2}"
    driver.get(url)
    driver.maximize_window()

    try:
        # Esperamos a que aparezca el filtro para que la pagina carge bien
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'a-filter--applied'))
        )
    except Exception as e:
        print(f"Ha habido un error esperando la página: {e}")
    
    sleep(1)

    contenido_html = driver.page_source
    sopa = BeautifulSoup(contenido_html, 'html.parser')
    
    driver.close()

    return sopa

In [156]:
def extraer_actividades_civitatis(ciudad, fecha1, fecha2):
    nombres = []
    puntuaciones = []
    precios = []
    descripciones = []
    links = []

    for pagina in tqdm(range(1, 7)):
        sopa = abrir_web(ciudad, pagina, fecha1, fecha2)
        categorias = sopa.findAll('div', {'class': 'o-search-list__item'})
        for categoria in categorias:
            
            try:
                nombres.append(categoria.find('h2', {"class": "comfort-card__title"}).text.strip())
            except:
                nombres.append(np.nan)
            try:
                puntuaciones.append(categoria.find('span', {"class": "m-rating--text"}).text.replace('/ 10', '').replace(',', '.').strip())    
            except:
                puntuaciones.append(np.nan)
            try:
                precios.append(categoria.find('span', {"class": "comfort-card__price__text"}).text.replace('€', '').strip())
            except:
                precios.append(np.nan)    
            try:
                descripciones.append(categoria.find('div', {"class": "comfort-card__text l-list-card__text"}).text.replace('\xa0',' ').strip())
            except:
                descripciones.append(np.nan)
            try:
                links.append(f"https://www.civitatis.com{categoria.find('a', {'class': 'ga-trackEvent-element _activity-link'}).get('href')}")
            except:
                links.append(np.nan)
        sleep(2)
    diccionario_categorias = {'Nombre': nombres, 'Puntuación': puntuaciones, 'Precio': precios, 'Descripción': descripciones, 'Link': links}
    
    return diccionario_categorias
        
    
    
        
    
    
    
    
    


In [157]:
def crear_df_actividades(ciudad, fecha1, fecha2):
    df_civitatis = pd.DataFrame(extraer_actividades_civitatis(ciudad, fecha1, fecha2))
    pd.set_option('display.max_colwidth', None)
    return df_civitatis

**Extraemos datos para Barcelona** 

In [158]:
df_barcelona = crear_df_actividades('barcelona', f'2024-11-22', f'2024-11-24')

100%|██████████| 6/6 [01:03<00:00, 10.58s/it]


In [159]:
df_santiago = crear_df_actividades('santiago-de-compostela', f'2024-11-22', f'2024-11-24')

100%|██████████| 6/6 [01:02<00:00, 10.42s/it]


In [160]:
df_barcelona

,Nombre,Puntuación,Precio,Descripción,Link
0,Visita de la Sagrada Familia sin colas,8.6,55,Si hay un lugar que nadie puede perderse en Barcelona es la Sagrada Familia. En esta visita descubriremos la obra maestra de Gaudí saltándonos sus colas.,https://www.civitatis.com/es/barcelona/visita-guiada-sagrada-familia/
1,Free tour por Barcelona,9.5,¡Gratis!,"El free tour por Barcelona es la mejor forma de explorar a pie la capital catalana. Conoceremos las calles, plazas y monumentos imprescindibles de la ciudad.",https://www.civitatis.com/es/barcelona/free-tour-barcelona/
2,Excursión a Montserrat con tren cremallera,8.4,57,"En esta excursión a Montserrat podréis visitar el Santuario de la Patrona de Cataluña, a 1236 metros de altura, y llegar en tren cremallera.",https://www.civitatis.com/es/barcelona/excursion-montserrat/
3,Visita guiada por el Parque Güell,7.7,"25,60",En esta visita guiada por el Parque Güell de Barcelona visitaremos una de las grandes joyas modernistas de Antonio Gaudí en la capital de Cataluña.,https://www.civitatis.com/es/barcelona/visita-guiada-parque-guell/
4,Traslados en Barcelona,NaN,46,"Servicio privado, puerta a puerta y al mejor precio. Un chófer os esperará en el aeropuerto, en el hotel o donde queráis para llevaros a vuestro destino. Rápido y seguro.",NaN
...,...,...,...,...,...
121,Excursión a Montserrat + Visita a una bodega,10,"79,90","En esta excursión a Montserrat no solo disfrutaremos de un tour por la abadía, sino que también degustaremos los vinos y tapas más famosos de Cataluña.",https://www.civitatis.com/es/barcelona/tour-tapas-vinos-montserrat/
122,Free tour de los misterios y leyendas del Barrio Gótico,9.4,¡Gratis!,Acompañadnos en este free tour por Barcelona en el que os contaremos los misterios y leyendas más divertidas del Barrio Gótico. ¡Lo pasaréis en grande!,https://www.civitatis.com/es/barcelona/free-tour-misterios-leyendas-barrio-gotico/
123,"Excursión a Gerona, Figueras y Museo Dalí",9,79,"En esta excursión a Gerona y Figueras conoceremos dos de las ciudades más especiales de Cataluña. Además, visitaremos el Museo Dalí, la obra magna del artista.",https://www.civitatis.com/es/barcelona/excursion-museo-dali-figueres/
124,Tour privado por Barcelona ¡Tú eliges!,9.4,160,"Reservando nuestra visita privada tendréis un guía para descubrir Barcelona en exclusiva con vuestra pareja, con tu familia o con tu grupo de amigos.",https://www.civitatis.com/es/barcelona/tour-privado-barcelona/


In [161]:
df_santiago

,Nombre,Puntuación,Precio,Descripción,Link
0,"Excursión a las Rías Baixas, isla de Arosa y Combarro",9.1,42,"Los paisajes de las Rías Baixas cautivan a todos. En esta excursión visitaremos bellos lugares como Padrón, Arosa, Cambados, La Toja y Combarro. ¡Os encantarán!",https://www.civitatis.com/es/santiago-de-compostela/rias-baixas-isla-arosa-combarro/
1,Free tour por Santiago de Compostela,9.3,¡Gratis!,"Recorre el centro histórico de Santiago de Compostela y descubre el conjunto monumental de esta ciudad Patrimonio de la Humanidad. Además, ¡es gratis!",https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-santiago-compostela/
2,Excursión a Finisterre y Costa da Morte de día completo,8.7,49,Dejaos cautivar por los paisajes del cabo Finisterre y la Costa da Morte en esta excursión de día completo por una de las zonas más bellas del litoral gallego.,https://www.civitatis.com/es/santiago-de-compostela/excursion-finisterre-costa-morte/
3,Visita guiada por la Catedral de Santiago,9.2,15,Descubrid uno de los iconos de la capital de Galicia en esta visita guiada por la Catedral de Santiago de Compostela. ¡Una actividad imprescindible!,https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-museo-catedral-santiago/
4,Excursión a las Rías Baixas + Visita a una bodega,8.9,42,"Una bodega de albariño, un paseo en barco por la Isla de La Toja, Combarro, la playa de La Lanzada… Hacer una excursión a las Rías Baixas es imprescindible.",https://www.civitatis.com/es/santiago-de-compostela/excursion-rias-baixas-visita-bodega/
...,...,...,...,...,...
127,Free tour de las mujeres ilustres de Santiago de Compostela,NaN,¡Gratis!,En este free tour de las mujeres ilustres de Santiago de Compostela conoceremos la historia de los personajes femeninos más importantes de la capital gallega.,https://www.civitatis.com/es/santiago-de-compostela/free-tour-mujeres-ilustres-santiago/
128,Tour gastronómico por Santiago de Compostela + Visita a la Catedral,9.2,65,"Los sabores de Galicia son únicos. Para comprobarlo, no hay nada mejor que hacer este tour gastronómico por Santiago de Compostela. ¡Sencillamente delicioso!",https://www.civitatis.com/es/santiago-de-compostela/tour-gastronomico-santiago/
129,Tour privado por la Catedral de Santiago y su museo,9.6,65,En este tour privado por la Catedral de Santiago descubriréis las maravillas de uno de los templos más importantes de España con un guía en exclusiva.,https://www.civitatis.com/es/santiago-de-compostela/tour-privado-catedral-santiago/
130,Free tour por el Santiago contemporáneo,9.3,¡Gratis!,"Si os apasiona la arquitectura y queréis descubrir una cara desconocida de la capital de Galicia, este free tour por el Santiago contemporáneo es para vosotros.",https://www.civitatis.com/es/santiago-de-compostela/free-tour-santiago-modernista/


In [162]:
df_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nombre       126 non-null    object
 1   Puntuación   120 non-null    object
 2   Precio       126 non-null    object
 3   Descripción  126 non-null    object
 4   Link         120 non-null    object
dtypes: object(5)
memory usage: 5.1+ KB


In [163]:
df_santiago.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nombre       132 non-null    object
 1   Puntuación   114 non-null    object
 2   Precio       132 non-null    object
 3   Descripción  132 non-null    object
 4   Link         126 non-null    object
dtypes: object(5)
memory usage: 5.3+ KB


Para el tratamiento de nulos, tenemos en cuenta que queremos dar la información más completa posible a nuestros clientes, en este sentido si no pueden acceder a la web (porque no hay link) a informarse por ellso mismos de las actividades creemos que es mejor prescindir de esa fila por entero. En cambio, las filas con nulos en la puntuación de los usuarios las dejamos, pues se trata de actividades de nueva incorporcación, y es normal que aun no haya puntuaciones. En el futuro las habrá y en todo caso, nuestros clientes pueden entrar en el link y recabar inforamción por ellos mismo de si quieren esa actividad o no.

In [164]:
df_barcelona = df_barcelona.dropna(subset=['Link'])

In [165]:
df_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 0 to 125
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nombre       120 non-null    object
 1   Puntuación   120 non-null    object
 2   Precio       120 non-null    object
 3   Descripción  120 non-null    object
 4   Link         120 non-null    object
dtypes: object(5)
memory usage: 5.6+ KB


In [166]:
df_santiago = df_santiago.dropna(subset=['Link'])

In [167]:
df_santiago.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 0 to 131
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nombre       126 non-null    object
 1   Puntuación   114 non-null    object
 2   Precio       126 non-null    object
 3   Descripción  126 non-null    object
 4   Link         126 non-null    object
dtypes: object(5)
memory usage: 5.9+ KB


In [168]:
#Sustituimos en nuestros dos datafrmaes los valores "¡Gratis!" de la columna precio por un 0
df_barcelona["Precio"] = df_barcelona["Precio"].replace('¡Gratis!', '0')
df_santiago["Precio"] = df_santiago["Precio"].replace('¡Gratis!', '0')


In [169]:
df_barcelona

,Nombre,Puntuación,Precio,Descripción,Link
0,Visita de la Sagrada Familia sin colas,8.6,55,Si hay un lugar que nadie puede perderse en Barcelona es la Sagrada Familia. En esta visita descubriremos la obra maestra de Gaudí saltándonos sus colas.,https://www.civitatis.com/es/barcelona/visita-guiada-sagrada-familia/
1,Free tour por Barcelona,9.5,0,"El free tour por Barcelona es la mejor forma de explorar a pie la capital catalana. Conoceremos las calles, plazas y monumentos imprescindibles de la ciudad.",https://www.civitatis.com/es/barcelona/free-tour-barcelona/
2,Excursión a Montserrat con tren cremallera,8.4,57,"En esta excursión a Montserrat podréis visitar el Santuario de la Patrona de Cataluña, a 1236 metros de altura, y llegar en tren cremallera.",https://www.civitatis.com/es/barcelona/excursion-montserrat/
3,Visita guiada por el Parque Güell,7.7,"25,60",En esta visita guiada por el Parque Güell de Barcelona visitaremos una de las grandes joyas modernistas de Antonio Gaudí en la capital de Cataluña.,https://www.civitatis.com/es/barcelona/visita-guiada-parque-guell/
5,Sagrada Familia con subida a las torres,7.8,70,En esta visita a la Sagrada Familia conoceremos el lugar más famoso de Barcelona y subiremos a una de sus torres. ¡Tendremos una vista inmejorable de la ciudad!,https://www.civitatis.com/es/barcelona/visita-sagrada-familia-torres/
...,...,...,...,...,...
121,Excursión a Montserrat + Visita a una bodega,10,"79,90","En esta excursión a Montserrat no solo disfrutaremos de un tour por la abadía, sino que también degustaremos los vinos y tapas más famosos de Cataluña.",https://www.civitatis.com/es/barcelona/tour-tapas-vinos-montserrat/
122,Free tour de los misterios y leyendas del Barrio Gótico,9.4,0,Acompañadnos en este free tour por Barcelona en el que os contaremos los misterios y leyendas más divertidas del Barrio Gótico. ¡Lo pasaréis en grande!,https://www.civitatis.com/es/barcelona/free-tour-misterios-leyendas-barrio-gotico/
123,"Excursión a Gerona, Figueras y Museo Dalí",9,79,"En esta excursión a Gerona y Figueras conoceremos dos de las ciudades más especiales de Cataluña. Además, visitaremos el Museo Dalí, la obra magna del artista.",https://www.civitatis.com/es/barcelona/excursion-museo-dali-figueres/
124,Tour privado por Barcelona ¡Tú eliges!,9.4,160,"Reservando nuestra visita privada tendréis un guía para descubrir Barcelona en exclusiva con vuestra pareja, con tu familia o con tu grupo de amigos.",https://www.civitatis.com/es/barcelona/tour-privado-barcelona/


In [170]:
df_santiago

,Nombre,Puntuación,Precio,Descripción,Link
0,"Excursión a las Rías Baixas, isla de Arosa y Combarro",9.1,42,"Los paisajes de las Rías Baixas cautivan a todos. En esta excursión visitaremos bellos lugares como Padrón, Arosa, Cambados, La Toja y Combarro. ¡Os encantarán!",https://www.civitatis.com/es/santiago-de-compostela/rias-baixas-isla-arosa-combarro/
1,Free tour por Santiago de Compostela,9.3,0,"Recorre el centro histórico de Santiago de Compostela y descubre el conjunto monumental de esta ciudad Patrimonio de la Humanidad. Además, ¡es gratis!",https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-santiago-compostela/
2,Excursión a Finisterre y Costa da Morte de día completo,8.7,49,Dejaos cautivar por los paisajes del cabo Finisterre y la Costa da Morte en esta excursión de día completo por una de las zonas más bellas del litoral gallego.,https://www.civitatis.com/es/santiago-de-compostela/excursion-finisterre-costa-morte/
3,Visita guiada por la Catedral de Santiago,9.2,15,Descubrid uno de los iconos de la capital de Galicia en esta visita guiada por la Catedral de Santiago de Compostela. ¡Una actividad imprescindible!,https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-museo-catedral-santiago/
4,Excursión a las Rías Baixas + Visita a una bodega,8.9,42,"Una bodega de albariño, un paseo en barco por la Isla de La Toja, Combarro, la playa de La Lanzada… Hacer una excursión a las Rías Baixas es imprescindible.",https://www.civitatis.com/es/santiago-de-compostela/excursion-rias-baixas-visita-bodega/
...,...,...,...,...,...
127,Free tour de las mujeres ilustres de Santiago de Compostela,NaN,0,En este free tour de las mujeres ilustres de Santiago de Compostela conoceremos la historia de los personajes femeninos más importantes de la capital gallega.,https://www.civitatis.com/es/santiago-de-compostela/free-tour-mujeres-ilustres-santiago/
128,Tour gastronómico por Santiago de Compostela + Visita a la Catedral,9.2,65,"Los sabores de Galicia son únicos. Para comprobarlo, no hay nada mejor que hacer este tour gastronómico por Santiago de Compostela. ¡Sencillamente delicioso!",https://www.civitatis.com/es/santiago-de-compostela/tour-gastronomico-santiago/
129,Tour privado por la Catedral de Santiago y su museo,9.6,65,En este tour privado por la Catedral de Santiago descubriréis las maravillas de uno de los templos más importantes de España con un guía en exclusiva.,https://www.civitatis.com/es/santiago-de-compostela/tour-privado-catedral-santiago/
130,Free tour por el Santiago contemporáneo,9.3,0,"Si os apasiona la arquitectura y queréis descubrir una cara desconocida de la capital de Galicia, este free tour por el Santiago contemporáneo es para vosotros.",https://www.civitatis.com/es/santiago-de-compostela/free-tour-santiago-modernista/


In [171]:
def convertir_columnas_a_float(df, columnas):
    """
    Convierte las columnas especificadas de un DataFrame de string a float.

    Parámetros:
    df (DataFrame): El DataFrame a modificar.
    columnas (list): Lista de nombres de las columnas que se desea convertir a float.

    Retorna:
    DataFrame: El DataFrame con las columnas convertidas.
    """
    for columna in columnas:
        # Intentamos convertir la columna a float, si falla se deja como NaN
        df[columna] = pd.to_numeric(df[columna], errors='coerce')
    
    return df


In [172]:
# Convertimos las columnas de Puntuación y Precio a float
convertir_columnas_a_float(df_barcelona, ["Puntuación", "Precio"])


,Nombre,Puntuación,Precio,Descripción,Link
0,Visita de la Sagrada Familia sin colas,8.6,55.0,Si hay un lugar que nadie puede perderse en Barcelona es la Sagrada Familia. En esta visita descubriremos la obra maestra de Gaudí saltándonos sus colas.,https://www.civitatis.com/es/barcelona/visita-guiada-sagrada-familia/
1,Free tour por Barcelona,9.5,0.0,"El free tour por Barcelona es la mejor forma de explorar a pie la capital catalana. Conoceremos las calles, plazas y monumentos imprescindibles de la ciudad.",https://www.civitatis.com/es/barcelona/free-tour-barcelona/
2,Excursión a Montserrat con tren cremallera,8.4,57.0,"En esta excursión a Montserrat podréis visitar el Santuario de la Patrona de Cataluña, a 1236 metros de altura, y llegar en tren cremallera.",https://www.civitatis.com/es/barcelona/excursion-montserrat/
3,Visita guiada por el Parque Güell,7.7,NaN,En esta visita guiada por el Parque Güell de Barcelona visitaremos una de las grandes joyas modernistas de Antonio Gaudí en la capital de Cataluña.,https://www.civitatis.com/es/barcelona/visita-guiada-parque-guell/
5,Sagrada Familia con subida a las torres,7.8,70.0,En esta visita a la Sagrada Familia conoceremos el lugar más famoso de Barcelona y subiremos a una de sus torres. ¡Tendremos una vista inmejorable de la ciudad!,https://www.civitatis.com/es/barcelona/visita-sagrada-familia-torres/
...,...,...,...,...,...
121,Excursión a Montserrat + Visita a una bodega,10.0,NaN,"En esta excursión a Montserrat no solo disfrutaremos de un tour por la abadía, sino que también degustaremos los vinos y tapas más famosos de Cataluña.",https://www.civitatis.com/es/barcelona/tour-tapas-vinos-montserrat/
122,Free tour de los misterios y leyendas del Barrio Gótico,9.4,0.0,Acompañadnos en este free tour por Barcelona en el que os contaremos los misterios y leyendas más divertidas del Barrio Gótico. ¡Lo pasaréis en grande!,https://www.civitatis.com/es/barcelona/free-tour-misterios-leyendas-barrio-gotico/
123,"Excursión a Gerona, Figueras y Museo Dalí",9.0,79.0,"En esta excursión a Gerona y Figueras conoceremos dos de las ciudades más especiales de Cataluña. Además, visitaremos el Museo Dalí, la obra magna del artista.",https://www.civitatis.com/es/barcelona/excursion-museo-dali-figueres/
124,Tour privado por Barcelona ¡Tú eliges!,9.4,160.0,"Reservando nuestra visita privada tendréis un guía para descubrir Barcelona en exclusiva con vuestra pareja, con tu familia o con tu grupo de amigos.",https://www.civitatis.com/es/barcelona/tour-privado-barcelona/


In [173]:
convertir_columnas_a_float(df_santiago, ["Puntuación", "Precio"])

,Nombre,Puntuación,Precio,Descripción,Link
0,"Excursión a las Rías Baixas, isla de Arosa y Combarro",9.1,42,"Los paisajes de las Rías Baixas cautivan a todos. En esta excursión visitaremos bellos lugares como Padrón, Arosa, Cambados, La Toja y Combarro. ¡Os encantarán!",https://www.civitatis.com/es/santiago-de-compostela/rias-baixas-isla-arosa-combarro/
1,Free tour por Santiago de Compostela,9.3,0,"Recorre el centro histórico de Santiago de Compostela y descubre el conjunto monumental de esta ciudad Patrimonio de la Humanidad. Además, ¡es gratis!",https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-santiago-compostela/
2,Excursión a Finisterre y Costa da Morte de día completo,8.7,49,Dejaos cautivar por los paisajes del cabo Finisterre y la Costa da Morte en esta excursión de día completo por una de las zonas más bellas del litoral gallego.,https://www.civitatis.com/es/santiago-de-compostela/excursion-finisterre-costa-morte/
3,Visita guiada por la Catedral de Santiago,9.2,15,Descubrid uno de los iconos de la capital de Galicia en esta visita guiada por la Catedral de Santiago de Compostela. ¡Una actividad imprescindible!,https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-museo-catedral-santiago/
4,Excursión a las Rías Baixas + Visita a una bodega,8.9,42,"Una bodega de albariño, un paseo en barco por la Isla de La Toja, Combarro, la playa de La Lanzada… Hacer una excursión a las Rías Baixas es imprescindible.",https://www.civitatis.com/es/santiago-de-compostela/excursion-rias-baixas-visita-bodega/
...,...,...,...,...,...
127,Free tour de las mujeres ilustres de Santiago de Compostela,NaN,0,En este free tour de las mujeres ilustres de Santiago de Compostela conoceremos la historia de los personajes femeninos más importantes de la capital gallega.,https://www.civitatis.com/es/santiago-de-compostela/free-tour-mujeres-ilustres-santiago/
128,Tour gastronómico por Santiago de Compostela + Visita a la Catedral,9.2,65,"Los sabores de Galicia son únicos. Para comprobarlo, no hay nada mejor que hacer este tour gastronómico por Santiago de Compostela. ¡Sencillamente delicioso!",https://www.civitatis.com/es/santiago-de-compostela/tour-gastronomico-santiago/
129,Tour privado por la Catedral de Santiago y su museo,9.6,65,En este tour privado por la Catedral de Santiago descubriréis las maravillas de uno de los templos más importantes de España con un guía en exclusiva.,https://www.civitatis.com/es/santiago-de-compostela/tour-privado-catedral-santiago/
130,Free tour por el Santiago contemporáneo,9.3,0,"Si os apasiona la arquitectura y queréis descubrir una cara desconocida de la capital de Galicia, este free tour por el Santiago contemporáneo es para vosotros.",https://www.civitatis.com/es/santiago-de-compostela/free-tour-santiago-modernista/


Veamos cuántas filas duplicadas tenemos

In [174]:
df_barcelona.duplicated().value_counts()

True     100
False     20
Name: count, dtype: int64

In [175]:
df_santiago.duplicated().value_counts()

True     105
False     21
Name: count, dtype: int64

Hay muchos duplicados en ambos DataFrames, cosa lógica, pues mucahs de las actividades se repiten alguno de los tres días. Como no hemos tenido en cuenta la fecha, pues dejamos que lo consulte el cliente a traves del link a la propia actividad que hay en cada fila del DataFrame, podemos, pues prescindir de los duplicados, y quedarnos solo con las actividades diferentes que habrá en cada una de las dos ciudades a lo largo de esos tres días.

In [176]:
df_barcelona=df_barcelona.drop_duplicates().reset_index(drop=True)
df_santiago=df_santiago.drop_duplicates().reset_index(drop=True)

Hemos concluido la limpieza de los dataframes. Los mostramos para ver como han quedado y los guardamos en un csv cada uno.

In [177]:
df_barcelona

,Nombre,Puntuación,Precio,Descripción,Link
0,Visita de la Sagrada Familia sin colas,8.6,55.0,Si hay un lugar que nadie puede perderse en Barcelona es la Sagrada Familia. En esta visita descubriremos la obra maestra de Gaudí saltándonos sus colas.,https://www.civitatis.com/es/barcelona/visita-guiada-sagrada-familia/
1,Free tour por Barcelona,9.5,0.0,"El free tour por Barcelona es la mejor forma de explorar a pie la capital catalana. Conoceremos las calles, plazas y monumentos imprescindibles de la ciudad.",https://www.civitatis.com/es/barcelona/free-tour-barcelona/
2,Excursión a Montserrat con tren cremallera,8.4,57.0,"En esta excursión a Montserrat podréis visitar el Santuario de la Patrona de Cataluña, a 1236 metros de altura, y llegar en tren cremallera.",https://www.civitatis.com/es/barcelona/excursion-montserrat/
3,Visita guiada por el Parque Güell,7.7,NaN,En esta visita guiada por el Parque Güell de Barcelona visitaremos una de las grandes joyas modernistas de Antonio Gaudí en la capital de Cataluña.,https://www.civitatis.com/es/barcelona/visita-guiada-parque-guell/
4,Sagrada Familia con subida a las torres,7.8,70.0,En esta visita a la Sagrada Familia conoceremos el lugar más famoso de Barcelona y subiremos a una de sus torres. ¡Tendremos una vista inmejorable de la ciudad!,https://www.civitatis.com/es/barcelona/visita-sagrada-familia-torres/
5,Free tour de Gaudí y la Barcelona modernista,9.7,0.0,Acompañadnos a visitar el corazón de Barcelona en este free tour de Gaudí que nos llevará a descubrir los principales símbolos del modernismo en España.,https://www.civitatis.com/es/barcelona/tour-barcelona-modernista/
6,Tour por la Sagrada Familia y el Parque Güell,7.9,89.0,"En este tour por Barcelona visitaremos la Sagrada Familia y el Parque Güell, las dos grandes joyas de la arquitectura modernista de Antonio Gaudí.",https://www.civitatis.com/es/barcelona/visita-parque-guell-sagrada-familia/
7,Excursión a Andorra y Aux Les Thermes,8.5,109.0,"¿Queréis conocer Andorra? En esta excursión desde Barcelona visitaremos Andorra la Vella, Pas de la Casa y el bonito pueblo francés de Aux Les Thermes.",https://www.civitatis.com/es/barcelona/excursion-andorra/
8,Tour de Barcelona al completo,8.6,NaN,"¿Queréis conocer lo mejor de Barcelona en un día? En este tour recorreremos la ciudad, subiremos al teleférico, pasearemos por el centro y por el Barrio Gótico.",https://www.civitatis.com/es/barcelona/tour-completo-barcelona/
9,Entrada a la Casa Batlló,9.3,35.0,Con la entrada a la Casa Batlló os sumergiréis en esta joya del modernismo y uno de los iconos de Barcelona. Conoceréis una de las obras más creativas de Gaudí.,https://www.civitatis.com/es/barcelona/entrada-casa-batllo/


In [178]:
df_santiago

,Nombre,Puntuación,Precio,Descripción,Link
0,"Excursión a las Rías Baixas, isla de Arosa y Combarro",9.1,42,"Los paisajes de las Rías Baixas cautivan a todos. En esta excursión visitaremos bellos lugares como Padrón, Arosa, Cambados, La Toja y Combarro. ¡Os encantarán!",https://www.civitatis.com/es/santiago-de-compostela/rias-baixas-isla-arosa-combarro/
1,Free tour por Santiago de Compostela,9.3,0,"Recorre el centro histórico de Santiago de Compostela y descubre el conjunto monumental de esta ciudad Patrimonio de la Humanidad. Además, ¡es gratis!",https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-santiago-compostela/
2,Excursión a Finisterre y Costa da Morte de día completo,8.7,49,Dejaos cautivar por los paisajes del cabo Finisterre y la Costa da Morte en esta excursión de día completo por una de las zonas más bellas del litoral gallego.,https://www.civitatis.com/es/santiago-de-compostela/excursion-finisterre-costa-morte/
3,Visita guiada por la Catedral de Santiago,9.2,15,Descubrid uno de los iconos de la capital de Galicia en esta visita guiada por la Catedral de Santiago de Compostela. ¡Una actividad imprescindible!,https://www.civitatis.com/es/santiago-de-compostela/visita-guiada-museo-catedral-santiago/
4,Excursión a las Rías Baixas + Visita a una bodega,8.9,42,"Una bodega de albariño, un paseo en barco por la Isla de La Toja, Combarro, la playa de La Lanzada… Hacer una excursión a las Rías Baixas es imprescindible.",https://www.civitatis.com/es/santiago-de-compostela/excursion-rias-baixas-visita-bodega/
5,Free tour de los misterios y leyendas de Santiago,9.4,0,En este free tour exploraremos los rincones más oscuros de Santiago y escucharemos sucesos que habrían hecho a más de un peregrino dar media vuelta.,https://www.civitatis.com/es/santiago-de-compostela/tour-teatralizado-misterios-leyendas/
6,Excursión a Finisterre y Mugía,9.2,36,En esta excursión a Finisterre y Mugía disfrutaremos de estas localidades de la Costa de la Muerte en un mismo día y visitaremos el mirador del Fin del Mundo.,https://www.civitatis.com/es/santiago-de-compostela/excursion-finisterre-mugia/
7,Tour de Santiago de Compostela al completo con entradas,9.5,45,"La plaza del Obradoiro, la Catedral, el Pórtico de la Gloria y el Hostal de los Reyes Católicos protagonizan este tour de Santiago de Compostela al completo.",https://www.civitatis.com/es/santiago-de-compostela/tour-santiago-compostela-completo/
8,Tour privado por Santiago ¡Tú eliges!,9.4,55,En este tour privado descubriréis el centro histórico de Santiago de Compostela con un guía en exclusiva que os mostrará los principales puntos turísticos.,https://www.civitatis.com/es/santiago-de-compostela/tour-privado-santiago/
9,Free tour por el Santiago oculto,9.5,0,Dejaos sorprender en este free tour por el Santiago oculto por las historias más increíbles de la ciudad mientras exploramos sus principales monumentos.,https://www.civitatis.com/es/santiago-de-compostela/free-tour-santiago-oculto/


In [179]:
df_barcelona.to_csv("datos/actividades_barcelona.csv", index = None)
df_santiago.to_csv("datos/actividades_santiago.csv", index = None)
